In [129]:
import numpy as np
import pandas as pd
import warnings
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


# Pré-processamento #

### Base e encoder manual ###

In [11]:
df = pd.read_csv(
    '../data/heart/processed/heart.csv',
    sep = ';', encoding = 'utf-8'
)

In [12]:
warnings.filterwarnings("ignore")

In [13]:
df_encod_manual = pd.DataFrame.copy(df)

In [14]:
df_encod_manual['Sex'].replace({
    'M': 0,
    'F': 1
}, inplace = True)

df_encod_manual['ChestPainType'].replace({
    'TA': 0,
    'ATA': 1,
    'NAP': 2,
    'ASY': 3
}, inplace = True)

df_encod_manual['RestingECG'].replace({
    'Normal': 0,
    'ST': 1,
    'LVH': 2
}, inplace = True)

df_encod_manual['ExerciseAngina'].replace({
    'N': 0,
    'Y': 1
}, inplace = True)

df_encod_manual['ST_Slope'].replace({
    'Up': 0,
    'Flat': 1,
    'Down': 2
}, inplace = True)

### Separação da base em previsores e classe alvo ###

In [15]:
previsores = df_encod_manual.iloc[:, 0:11].values

In [16]:
alvo = df_encod_manual.iloc[:, 11].values

### Escalonamento ###

In [17]:
previsores_esc = StandardScaler().fit_transform(previsores)

In [18]:
previsores_df = pd.DataFrame(previsores_esc)

#### LabelEncoder ####

In [19]:
previsores_label = df.iloc[:, 0:11].values

In [20]:
previsores_label[:, 1] = LabelEncoder().fit_transform(previsores[:, 1])

In [21]:
previsores_label[:, 2] = LabelEncoder().fit_transform(previsores_label[:, 2])
previsores_label[:, 6] = LabelEncoder().fit_transform(previsores_label[:, 6])
previsores_label[:, 8] = LabelEncoder().fit_transform(previsores_label[:, 8])
previsores_label[:, 10] = LabelEncoder().fit_transform(previsores_label[:, 10])

#### OneHotEncoder ####

In [22]:
previsores_hot = ColumnTransformer(
    transformers = [('OneHot', OneHotEncoder(), [1, 2, 6, 8, 10])],
    remainder = 'passthrough'
).fit_transform(previsores_label)

In [23]:
previsores_hot_df = pd.DataFrame(previsores_hot)

#### OneHot + Escalonamento ####

In [24]:
previsoresHot_esc = StandardScaler().fit_transform(previsores_hot)

In [25]:
previsoresHot_esc_df = pd.DataFrame(previsoresHot_esc)

## Separação dos dados em treino e teste ##
<span style="font-size: small;"> 
- <strong>arrays:</strong> nomes dos atributos previsores e alvo.</br>
- <strong>test_size:</strong> tamanho em porcentagem dos dados de teste. default é none. </br> 
- <strong>train_size:</strong> tamanho em porcentagem dos dados de treinamento.default é none. </br>  
- <strong>random_state:</strong> nomeação de um estado aleatório. </br>
- <strong>shuffle:</strong> embaralhamento dos dados aleatórios. Associado com o random_state ocorre o mesmo embaralhamento sempre. Default é True. </br>
- <strong>stratify:</strong> Possibilidade de dividir os dados de forma estratificada. Default é None (nesse caso é mantido a proporção, isto é, se tem 30% de zeros e 70% de 1 no dataframe, na separação em treinamento e teste se manterá essa proporção). </span>

In [110]:
x_train, x_test, y_train, y_test = train_test_split(
    previsoresHot_esc, alvo,
    test_size = 0.3, random_state = 0
)

# Criando o Algoritmo #
<span style="font-size: 14px;">
O algoritmo Naive Bayes é um método de classificação muito popular em Machine Learning. Ele é baseado no Teorema de Bayes e é frequentemente usado em problemas de classificação de texto, como categorização de emails, análise de sentimentos em redes sociais, entre outros. Neste documento, vamos explorar como o Naive Bayes funciona e como ele é aplicado na prática.
</br>

**Teorema de Bayes** 
O Naive Bayes utiliza o Teorema de Bayes para calcular a probabilidade de uma classe (por exemplo, "positivo" ou "negativo") dado um conjunto de características (por exemplo, palavras em um texto, características de um objeto, etc.). O Teorema de Bayes é definido como:
- P(A∣B)= P(B∣A)⋅P(A) / P(B) </br>
Onde:
- \( P(A|B) \) é a probabilidade de A dado B.
- \( P(B|A) \) é a probabilidade de B dado A.
- \( P(A) \) e \( P(B) \) são as probabilidades marginais de A e B, respectivamente.
</br>

**Suposição "Naive"**
O termo "naive" (ingênuo) vem da suposição feita pelo algoritmo de que as características são independentes umas das outras, ou seja, a presença de uma característica não afeta a presença de outra. Embora essa suposição seja raramente verdadeira na prática, ela simplifica muito os cálculos e, surpreendentemente, muitas vezes funciona bem na prática.
</br>

**Treinamento e Classificação**
Durante a fase de treinamento, o Naive Bayes calcula as probabilidades condicionais das características para cada classe. Isso significa que ele calcula a probabilidade de cada característica ocorrer em cada classe. Quando um novo exemplo é apresentado ao modelo, o Naive Bayes calcula a probabilidade de pertencer a cada classe com base nas probabilidades condicionais calculadas durante o treinamento e na suposição ingênua de independência entre as características. O exemplo é então classificado na classe com a maior probabilidade.
</br>

**Variantes do Naive Bayes**
Existem diferentes variantes do algoritmo Naive Bayes, como Bernoulli Naive Bayes, Multinomial Naive Bayes e Gaussian Naive Bayes. Cada uma dessas variantes é adequada para diferentes tipos de dados e distribuições de probabilidade.
</br>

**Conclusão**
O algoritmo Naive Bayes é uma técnica simples e poderosa de classificação que se baseia no Teorema de Bayes e na suposição ingênua de independência entre as características. Apesar de sua simplicidade, o Naive Bayes é frequentemente usado com sucesso em uma variedade de aplicações de Machine Learning.
</span>

In [111]:
naive = GaussianNB()
naive.fit(x_train, y_train);

## Avaliação do algoritmo ##

In [112]:
previsoes_test = naive.predict(x_test)

In [113]:
previsoes_test

array([1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1], dtype=int64)

In [114]:
previsoes_train = naive.predict(x_train)

### Métricas ###
Dados de teste: avaliado em um conjunto separado de dados que não foram vistos durante o treinamento.</br>
Dados de treino: avaliado nos mesmos dados que foram utilizados para treiná-lo.

**Dados de Teste**

In [115]:
accuracy_test = accuracy_score(y_test, previsoes_test) * 100.0

In [116]:
print("Acurácia do modelo em dados de teste: %.2f%%" % accuracy_test)

Acurácia do modelo em dados de teste: 84.78%


In [117]:
conf_matrix_test = confusion_matrix(y_test, previsoes_test)
print("Matriz de confusão do modelo em dados de teste:\n", conf_matrix_test)

Matriz de confusão do modelo em dados de teste::
 [[100  21]
 [ 21 134]]


In [118]:
classif_report_test = classification_report(y_test, previsoes_test)
print("Relatório do modelo em dados de teste: \n", classif_report_test)

Relatório do modelo em dados de teste:: 
               precision    recall  f1-score   support

           0       0.83      0.83      0.83       121
           1       0.86      0.86      0.86       155

    accuracy                           0.85       276
   macro avg       0.85      0.85      0.85       276
weighted avg       0.85      0.85      0.85       276



**Dados de treino**

In [119]:
accuracy_train = accuracy_score(y_train, previsoes_train) * 100.0

In [120]:
print("Acurácia do modelo em dados de treino: %.2f%%" % accuracy_train)

Acurácia do modelo em dados de treino: 86.12%


In [121]:
conf_matrix_train = confusion_matrix(y_train, previsoes_train)
print("Matriz de confusão do modelo em dados de treino:\n", conf_matrix_train)

Matriz de confusão do modelo em dados de treino:
 [[248  41]
 [ 48 304]]


In [122]:
classif_report_train = classification_report(y_train, previsoes_train)
print("Relatório do modelo em dados de treino: \n", classif_report_train)

Relatório do modelo em dados de treino: 
               precision    recall  f1-score   support

           0       0.84      0.86      0.85       289
           1       0.88      0.86      0.87       352

    accuracy                           0.86       641
   macro avg       0.86      0.86      0.86       641
weighted avg       0.86      0.86      0.86       641



## Validação cruzada ##

In [130]:
## Separando os dados em folds
kfold = KFold(
    n_splits = 30, shuffle = True, random_state = 5
)

In [131]:
## Criando o modelo
modelo = GaussianNB()
resultado = cross_val_score(
    modelo, previsoresHot_esc, alvo, cv = kfold
)

In [132]:
## Média e desvio padrão
print("Acurácia Média: %.2f%%" %(resultado.mean() * 100.0))

Acurácia Média: 85.17%
